<a href="https://colab.research.google.com/github/lucasbona88/mylims-consumiveis/blob/main/consulta_precos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importações e definições globais.

In [ ]:
import requests
import json
import csv
from datetime import date
from google.colab import userdata

"""
Valiáveis importantes (contrato deve ser feito diretamente com Labsoft)
Há um jeito de fazer o login com usuários normais, porém usa forma de Login e Logout
"""
tk = userdata.get('token')
lab = userdata.get('lab')
cabecalho = {'x-access-key': tk, 'token': tk}

Definição de funções importantes para todo o sistema.

In [ ]:
def get_json(url, id):
  """
  Função principal para retornar todos os dados no myLIMS através das URL:
  url_detalhes = retorna informações e os reagentes que foram usados para criar a solução.
  url_lista = retorna o preço do reagente, unidade de medida e quantidade em estoque, se necessário.
  url_mov = retorna informações de informação de consumível, útil para saber quanta água foi usada para avolumar.
  """
  url_detalhes = f"https://{lab}.mylimsweb.cloud/api/ConsumableInfoApi/GetByConsumableId?consumableId={id}&includeProperties=Info%2CInfoType%2CInfo.Options%2CMeasurementUnit%2CValueEquipment%2CValueAccount%2CValueConsumableMovement%2CValueFile%2CDependentInfo"
  url_lista = f"https://{lab}.mylimsweb.cloud/api/v2/Consumables/ByServiceCenter?%24inlinecount=allpages&%24top=20&%24orderby=Id+desc&%24filter=Id+eq+{id}"
  url_mov = f"https://{lab}.mylimsweb.cloud/api/ConsumableMovementApi/GetConsumableMovements?consumableId={id}&%24inlinecount=allpages&%24top=25"
  if url == "url_lista":
    r = requests.get(url_lista, headers= cabecalho)
  elif url == "url_detalhes":
    r = requests.get(url_detalhes, headers= cabecalho)
  else:
    r = requests.get(url_mov, headers= cabecalho)
  to_json = json.loads(r.content)
  return to_json

Definições do Objeto Reagente

In [ ]:
# definições da classe
class Reagente:
  def __init__(self, id: int, identificacao="", preco="-", estoque="-", unidade="", consumo =""):
    self.id = id
    self.identificacao = identificacao
    self.preco = preco
    self.estoque = estoque
    self.unidade = unidade
    self.consumo = consumo
    self.reagente_id = []
    self.reagente_unidade = []
    self.reagente_preco = []
    self.reagente_consumo = []
    self.quantidade = 0

    self.consulta_dados(self.id)
    self.get_quantidade(self.id)

  def mostra(self):
    """
    Retornar uma frase com as principais informações
    """
    return f'ID: {self.id}, Identificação: {self.identificacao}, Preço: {self.preco}, Unidade: {self.unidade}, Consumo: {self.consumo}, Reagentes: {self.reagente_id}, Quantidade Produzida: {self.quantidade}'

  def get_quantidade(self, id):
    """
    Retorna a quantidade de reagente cadastrada
    """
    to_json = get_json(url="url_mov", id=id)
    self.quantidade = to_json['Items'][0]['Quantity']

  def consulta_dados(self, id):
    """
    Busca na 'url_lista'
    Retorna dados do Reagente, se o preço existir preenche o mesmo.
    Dados que retornam:
    - Unidade;
    - Identificação;
    - Preço.
    """
    try:
      to_json = get_json(url="url_lista", id=id)
      if to_json['Count'] == 0:
        raise KeyError
      else:
        self.unidade = to_json['Result'][0]['ConsumableType']['MeasurementUnit']['Identification']
        self.identificacao = to_json['Result'][0]['ConsumableType']['Identification']
        self.preco = to_json['Result'][0]['Cost']
        self.estoque = to_json['Result'][0]['Quantity']
    except KeyError as e:
      print(f"Error retrieving data for ID {id}: {e}")
      self.unidade = None

  def get_consumiveis(self):
    """
    Faz uma busca na 'url_detalhes' pelo ID do consumível principal.
    Retorna os IDs, unidade de medida e quantidade utilizada.
    O filtro para saber qual informação é um consumível é se é um tipo de informação = 8 (movimento de consumível).
    """
  # pegando dados dos consumíveis
    # print(f'Pegando dados de consumíveis do ID:{self.id}')
    to_json = get_json(url="url_detalhes", id=self.id)
    #  Pegar os consumíveis
    # print(f'aqui o to_json {to_json}')
    for item in to_json:
      if item['Info']['InfoTypeId'] == 8:
        if item['ValueConsumableMovement'] == None:
          self.reagente_id.append(0)
          self.reagente_unidade.append(0)
          self.reagente_consumo.append(1)
        else:
          if item['ValueConsumableMovement']['Quantity'] == 0:
            self.reagente_id.append(item['ValueConsumableMovement']['ConsumableId'])
            self.reagente_unidade.append(item['MeasurementUnit']['Identification'])
            self.reagente_consumo.append(1)
          else:
            self.reagente_id.append(item['ValueConsumableMovement']['ConsumableId'])
            self.reagente_unidade.append(item['DisplayValue'])
            self.reagente_consumo.append((item['ValueConsumableMovement']['Quantity'] *-1))
            self.reagente_preco.append(get_preco(item['ValueConsumableMovement']['ConsumableId']))
      else:
        pass

  def registra_dados(self, nome_arquivo):
    """
    Exporta as informações do reagente para um arquivo CSV.
    Args:
        nome_arquivo (str): Nome do arquivo CSV a ser criado.
    """
    nome_arquivo = f"/content/drive/MyDrive/consulta_precos/" + nome_arquivo
    with open(nome_arquivo, "a", newline="",encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter =";",dialect= 'excel')
        if csv_file.tell() == 0:  # Cabeçalho do CSV
            writer.writerow(["ID", "Identificacao", "Unidade", "ID R.", "Qtd. R. ","Unid R.",  "Lote R.", "Custo Reagentes", "Quantidade Frasco", "Custo Unidade"])
        if self.reagente_id == []:
          dados = [self.id, self.identificacao, self.unidade, f'-', f'-', f'-', '-', '-', str(self.quantidade).replace(".",","), str(self.preco).replace(".",",") ]
          writer.writerow(dados)
        else:
          for i in range(len(self.reagente_id)):
            dados = [self.id, self.identificacao, self.unidade, self.reagente_id[i], str(self.reagente_consumo[i]).replace(".",","),
                    str(self.reagente_unidade[i]).split(' ')[1],self.reagente_unidade[i], str(self.reagente_preco[i]).replace(".",","), str(self.quantidade).replace(".",","), '-']
            writer.writerow(dados)
        # print(f'Registrando no arquivo: {nome_arquivo}')
    nome_arquivo = None


  def registra_estoque(self, nome_arquivo):
    """
    Exporta as informações de estoque dos reagentes.
    Args:
        nome_arquivo (str): Nome do arquivo CSV a ser criado.
    """
    nome_arquivo = f"/content/drive/MyDrive/mapa_PF/" + nome_arquivo
    data_consulta = date.today()
    with open(nome_arquivo, "a", newline="",encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter =";",dialect= 'excel')
        if csv_file.tell() == 0:  # Cabeçalho do CSV
            writer.writerow(["ID", "Identificacao", "Estoque", "Unidade", "Data"])
        dados = [self.id, self.identificacao, str(self.estoque).replace(".",","), self.unidade, data_consulta ]
        # print(f'Registrando no arquivo: {nome_arquivo}')
        writer.writerow(dados)
    nome_arquivo = None

Definição de outras Funções auxiliares

In [ ]:
def get_preco(id):
  """
  Retorna o preço a partir de um id buscando diretamente na 'url_lista', caso não exista preço
  retorna com '-'
  """
  # print(f'tentando pegar preço do id {id}')
  to_json = get_json(url="url_lista", id=id)

  preco = to_json['Result'][0]['Cost']
  # print(f'Preço encontrado: {preco}')
  if preco == None:
    # print(f'{preco} - categorizado como None')
    return "-"
  else:
    # print(f'{preco} - categorizado como número')
    return preco

def pega_todos_precos(lista_id = []):
  """
  Basicamente utiliza a função get_preco() para uma lista de ids.
  """
  # print('iniciando pega_todos_precos()')
  lista_preco = []
  length = len(lista_id)
  # print(f'lista de ids passado: {lista_id}')
  for i in range(length):
    if lista_id[i] == 0:
      lista_preco.append(0)
    else:
      lista_preco.append(get_preco(lista_id[i]))
  return lista_preco

def calcula_preco(quantidade, lista_id = [],lista_preco = [], lista_quant = []):
  """
  Com a lista de IDs, quantidades e preços faz o cálculo de custo de reagentes.
  Chama automaticamente a função pega_todos_precos() para preencher a lista de preços.
  Retorna o cálculo de preço de um reagente.
  """
  # print('Pegando todos os preços')
  preco_final = 0
  lista_preco = pega_todos_precos(lista_id)
  length = len(lista_id)
  novo_reagente = {}
  # if len(lista_id) == 0:
  #   raise ValueError("lista_id is empty")
  for i in range(length):
    # print('Início dos cálculos')
    if lista_id[i] == 0:
      lista_preco[i] = 0
    elif lista_preco[i] == "-":
      # cria um Reagente com este id
      # print('Criando um novo reagente')
      novo_reagente[i] = lista_id[i]

      novo_reagente[i] = Reagente(id=int(novo_reagente[i]))
      # print(f'id do novo_reagente = {novo_reagente[i].id}')
      novo_reagente[i].get_consumiveis()
      lista_preco[i] = float(calcula_preco(novo_reagente[i].quantidade, novo_reagente[i].reagente_id, novo_reagente[i].reagente_preco, novo_reagente[i].reagente_consumo))
      # print(f'foi pelo calcula preco {lista_preco[i]}')
      # print('passou direto')

    preco_final = preco_final + ( lista_preco[i] * lista_quant[i])
    # print(f'Calculando preço: {preco_final}')
  # print(f'Preço Final: {preco_final}')
  preco_final = round(preco_final, 2)
  return preco_final

def calcula_e_registra(id, saida="saida.csv"):
  """
  Com um id de qualquer consumível do myLIMS, registra em um csv 'saida.csv' ou outro.
  Caso não seja um Reagente com preço, faz o cálculo
  """
  # print('Iniciando calcula_e_registra()')
  alvo = Reagente(id=id)
  # print(alvo.mostra())
  if alvo.preco == None:
    # print('passo 1')
    alvo.get_consumiveis()

    alvo.preco = calcula_preco(alvo.quantidade, alvo.reagente_id, [], alvo.reagente_consumo)

  alvo.registra_dados(saida)

def get_registra_estoque(id, saida="saida.csv"):
  """
  Com um id de qualquer consumível do myLIMS, registra em um csv 'saida.csv' ou outro.
  Registra o estoque do id passado
  """
  alvo = Reagente(id=id)
  alvo.consulta_dados(id=alvo.id)
  alvo.registra_estoque(saida)

Teste com uma lista de IDS com produtos controlados

In [ ]:
# lista de IDs dos reagentes controlados
controlados = [11615,1255,4559,11640,1254,3589,3683,5332,1260,1681,5995,1556,5992,6031,13747,16040,5412,10199,9989,1519,10924,7990,13897,13901,4549,5996,5134,1378,4555,755,4556,4551,3670,16046,5404,10193,15894,16038,6041]

for reagente in controlados:
  get_registra_estoque(reagente)

Através de um arquivo csv com lista de IDS de reagentes, pega um por um e gera outro arquivo com os valores calculados.

In [ ]:
# Nome do arquivo CSV
arquivo_entrada = f"/content/drive/MyDrive/consulta_precos/" + 'lista.csv'

# Abre arquivo csv com lista de reagentes a pesquisar
with open(arquivo_entrada, newline='') as csvfile:
  reader = csv.reader(csvfile)
  read = list(reader)

  for linha in read[1:]:
    id = int(linha[0])
    calcula_e_registra(id=id, saida="saida.csv")

In [ ]:
calcula_e_registra(15609)